In [46]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [47]:
user_data = [
    {
        "User": "User1",
        "Theme": ["romantic_date"],
        "Other": ["Korean food", "Spa"],
        "Budget": 2
    },
    {
        "User": "User2",
        "Theme": ["night_out"],
        "Other": ["Italian food", "shopping"],
        "Budget": 3
    },
    {
        "User": "User3",
        "Theme": ["outdoor_adventure"],
        "Other": ["Golf", "Landmark"],
        "Budget": 4
    },
    {
        "User": "User4",
        "Theme": ["relaxation_and_wellness"],
        "Other": ["Spas", "Museum"],
        "Budget": 3
    }

]

# Creating a DataFrame for user profiles
df_user_profile = pd.DataFrame(user_data)
df_user_profile['combined_preferences'] = df_user_profile.apply(lambda row: row['Theme'] + row['Other'], axis=1)

# Print the user profile DataFrame
df_user_profile.head()


,User,Theme,Other,Budget,combined_preferences
0,User1,[romantic_date],"[Korean food, Spa]",2,"[romantic_date, Korean food, Spa]"
1,User2,[night_out],"[Italian food, shopping]",3,"[night_out, Italian food, shopping]"
2,User3,[outdoor_adventure],"[Golf, Landmark]",4,"[outdoor_adventure, Golf, Landmark]"
3,User4,[relaxation_and_wellness],"[Spas, Museum]",3,"[relaxation_and_wellness, Spas, Museum]"


In [48]:
# Sample user ratings (User-Item interactions)
ratings_data = {
    "User": ["User1", "User1", "User2", "User3", "User4", "User4", "User3", "User2"],
    "name": ["Potomac Pizza", "SeoulSpice", "Mamma Lucia", "National Archives archeological site", "Pebbles Wellness Spa", "Looney's Pub", "University of Maryland Golf Course", "The Cornerstone Grill & Loft"],
    "Rating": [5, 2, 1, 2, 2, 3, 2, 2]
}

# Creating a DataFrame for ratings
ratings_df = pd.DataFrame(ratings_data)

# Print the ratings DataFrame
ratings_df.head()


,User,name,Rating
0,User1,Potomac Pizza,5
1,User1,SeoulSpice,2
2,User2,Mamma Lucia,1
3,User3,National Archives archeological site,2
4,User4,Pebbles Wellness Spa,2


In [49]:

places_data = [
  {
    "name": "Potomac Pizza",
    "address": "7777 Baltimore Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9873178,
      "lon": -76.9356036
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "The Spa at The Hotel at the University of Maryland",
    "address": "7777 Baltimore Ave, FL 4, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9869586,
      "lon": -76.935151
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "The Hall CP",
    "address": "4656 Hotel Drive, College Park, MD 20742, United States",
    "location": {
      "lat": 38.9861878,
      "lon": -76.9336134
    },
    "category": "Restaurant",
    "category2": "Nightlife"
  },
  {
    "name": "Dog Haus Biergarten",
    "address": "7401 Baltimore Ave, Unit 1A, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9812184,
      "lon": -76.9375926
    },
    "category": "Nightlife",
    "category2": "Nightlife"
  },
  {
    "name": "Terrapins Turf",
    "address": "4410 Knox Rd, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9810151,
      "lon": -76.9384146
    },
    "category": "Nightlife",
    "category2": "Nightlife"
  },
  {
    "name": "The Cornerstone Grill & Loft",
    "address": "7325 Baltimore Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9806529,
      "lon": -76.9375442
    },
    "category": "Nightlife",
    "category2": "Nightlife"
  },
  {
    "name": "College Park Shopping Center",
    "address": "7370 Baltimore Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9806676,
      "lon": -76.9390872
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Looney's Pub",
    "address": "8150 Baltimore Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9914766,
      "lon": -76.9343682
    },
    "category": "Nightlife",
    "category2": "Nightlife"
  },
  {
    "name": "SeoulSpice",
    "address": "4200 Guilford Dr, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9804121,
      "lon": -76.942514
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "MeatUp Korean BBQ & Bar",
    "address": "8503 Baltimore Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9948577,
      "lon": -76.9319577
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Pebbles Wellness Spa",
    "address": "8507 Baltimore Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9953574,
      "lon": -76.9317421
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Paint Branch Golf Complex",
    "address": "4690 University Blvd, College Park, MD  20740, United States",
    "location": {
      "lat": 39.0038245,
      "lon": -76.93561
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "University of Maryland Golf Course",
    "address": "3800 Golf Course Rd, College Park, MD 20742, United States",
    "location": {
      "lat": 38.9910788,
      "lon": -76.9548726
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Bonchon",
    "address": "6507 America Blvd, Ste 101, Hyattsville, MD  20782, United States",
    "location": {
      "lat": 38.9686852,
      "lon": -76.9517612
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Calvert Road Disc Golf Course",
    "address": "5201 Paint Branch Pkwy, College Park, MD 20740, United States",
    "location": {
      "lat": 38.9756263,
      "lon": -76.9167584
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "The Shoppes At Metro Station",
    "address": "6211 Belcrest Rd, Hyattsville, MD  20782, United States",
    "location": {
      "lat": 38.9655546,
      "lon": -76.9532033
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Riversdale House Museum",
    "address": "4811 Riverdale Rd, Riverdale, MD  20737, United States",
    "location": {
      "lat": 38.9605855,
      "lon": -76.9313344
    },
    "category": "Museum",
    "category2": "Historical Sites"
  },
  {
    "name": "National Archives archeological site",
    "address": "8601 Adelphi Rd, College Park, MD  20742, United States",
    "location": {
      "lat": 39.000308,
      "lon": -76.959666
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "Mall at Prince George's",
    "address": "3500 East West Hwy, Hyattsville, MD 20782, United States",
    "location": {
      "lat": 38.9683307,
      "lon": -76.9574207
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Olive Garden Italian Restaurant",
    "address": "3480 East-West Hwy, Hyattsville, MD  20782, United States",
    "location": {
      "lat": 38.967275,
      "lon": -76.9585524
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "Iron Pig Korean BBQ",
    "address": "6107 Greenbelt Rd, Berwyn Heights, MD  20740, United States",
    "location": {
      "lat": 38.9969666,
      "lon": -76.9089081
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Essential Day Spa 2",
    "address": "5331 Baltimore Ave, Ste 106, Hyattsville, MD  20781, United States",
    "location": {
      "lat": 38.9547558,
      "lon": -76.9400233
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Mamma Lucia",
    "address": "4734 Cherry Hill Rd, College Park, MD  20740, United States",
    "location": {
      "lat": 39.016106,
      "lon": -76.928265
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "Beltway Plaza Mall",
    "address": "6000 Greenbelt Rd, Greenbelt, MD 20770, United States",
    "location": {
      "lat": 38.9994255,
      "lon": -76.908567
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Three Brothers Italian Restaurant",
    "address": "6160 Greenbelt Rd, Greenbelt, MD  20770, United States",
    "location": {
      "lat": 38.9994867,
      "lon": -76.9085112
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "Hollywood Shopping Center",
    "address": "9801 Rhode Island Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 39.0143196,
      "lon": -76.9196925
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Essential Day Spa",
    "address": "10250 Baltimore Ave\nUnit D\nCollege Park, MD  20740\nUnited States",
    "location": {
      "lat": 39.022529,
      "lon": -76.9259158
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Go Kart Track",
    "address": "4300 Kenilworth Ave, Bladensburg, MD 20710, United States",
    "location": {
      "lat": 38.9442694,
      "lon": -76.9327497
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "Parkway Spa Asian Massage",
    "address": "5507 Landover Rd, Hyattsville, MD 20784, United States",
    "location": {
      "lat": 38.939225,
      "lon": -76.9224179
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Hillandale Shopping Center",
    "address": "1620 Elton Rd, Silver Spring, MD  20903, United States",
    "location": {
      "lat": 39.0213229,
      "lon": -76.9752482
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Myong Dong",
    "address": "11114 Baltimore Ave, Beltsville, MD  20705, United States",
    "location": {
      "lat": 39.035736,
      "lon": -76.9075025
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Greenway Shopping Center",
    "address": "7547 Greenbelt Rd, Greenbelt, MD  20770, United States",
    "location": {
      "lat": 38.9922394,
      "lon": -76.8786699
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Da Rae Won Restaurant",
    "address": "5013 Garrett Ave, Beltsville, MD  20705, United States",
    "location": {
      "lat": 39.041095,
      "lon": -76.9064398
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Gah Rham Korean BBQ Restaurant",
    "address": "5027 Garrett Ave, Beltsville, MD  20705, United States",
    "location": {
      "lat": 39.0410352,
      "lon": -76.9056434
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Bryant Street NE",
    "address": "680 Rhode Island Ave NE, Washington, DC  20002, United States",
    "location": {
      "lat": 38.9220672,
      "lon": -76.9969164
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Firepan Korean BBQ",
    "address": "962 Wayne Ave, Silver Spring, MD  20910, United States",
    "location": {
      "lat": 38.9942451,
      "lon": -77.025975
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Langston Golf Course",
    "address": "2600 Benning Rd NE, Washington, DC  20002, United States",
    "location": {
      "lat": 38.9008777,
      "lon": -76.9674435
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Sligo Creek Golf Course",
    "address": "9701 Sligo Creek Pkwy, Silver Spring, MD  20901, United States",
    "location": {
      "lat": 39.0139932,
      "lon": -77.0262775
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Granville Moore's",
    "address": "1238 H St NE, Washington, DC  20002, United States",
    "location": {
      "lat": 38.9004267,
      "lon": -76.988931
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "Gunpowder Golf Course",
    "address": "14300 Old Gunpowder Rd, Laurel, MD 20707, United States",
    "location": {
      "lat": 39.0864269,
      "lon": -76.9201451
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Rock Creek Park Golf Course",
    "address": "6100 16th St NW, Washington, DC  20011, United States",
    "location": {
      "lat": 38.966433,
      "lon": -77.0407677
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Iron Age Korean Steak House",
    "address": "3365 14th St NW, Washington, DC  20010, United States",
    "location": {
      "lat": 38.9311633,
      "lon": -77.032367
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "9:30 Club",
    "address": "815 V St NW, Washington, DC  20001, United States",
    "location": {
      "lat": 38.9178869,
      "lon": -77.0237142
    },
    "category": "MusicVenue",
    "category2": "Nightlife"
  },
  {
    "name": "Morgan Black Spas",
    "address": "10111 Martin Luther King Jr Hwy, Bowie, MD  20720, United States",
    "location": {
      "lat": 38.9553445,
      "lon": -76.8287839
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Caruso's Grocery",
    "address": "914 14th St SE, Washington, DC  20003, United States",
    "location": {
      "lat": 38.879753,
      "lon": -76.985214
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "The Little Gay Pub",
    "address": "1100 P St NW, Washington, DC 20001, United States",
    "location": {
      "lat": 38.9095139,
      "lon": -77.0273898
    },
    "category": "Nightlife",
    "category2": "Nightlife"
  },
  {
    "name": "Trattoria Alberto's Italian Cuisine",
    "address": "506 Eighth St SE, Washington, DC  20003, United States",
    "location": {
      "lat": 38.8824609,
      "lon": -76.994699
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "RPM Italian",
    "address": "650 K Street NW, Washington, DC 20001, United States",
    "location": {
      "lat": 38.9021968,
      "lon": -77.0209301
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "Belga Cafe",
    "address": "514 Eighth St SE, Washington, DC  20003, United States",
    "location": {
      "lat": 38.8821764,
      "lon": -76.9947356
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "United States Capitol",
    "address": "Washington, DC  20001, United States",
    "location": {
      "lat": 38.8894251,
      "lon": -77.008616
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "CityCenterDC",
    "address": "825 Tenth St NW, Washington, DC 20001, United States",
    "location": {
      "lat": 38.9004601,
      "lon": -77.0255113
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Etalon Day Spa",
    "address": "707 D St NW, Washington, DC  20004, United States",
    "location": {
      "lat": 38.8949455,
      "lon": -77.0223134
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Anju",
    "address": "1805 18th St NW, Washington, DC  20009, United States",
    "location": {
      "lat": 38.9143441,
      "lon": -77.0414731
    },
    "category": "Restaurant",
    "category2": "Korean food"
  },
  {
    "name": "Aura spa - Yards",
    "address": "1212 Fourth St SE, Ste 170, Washington, DC  20003, United States",
    "location": {
      "lat": 38.8753353,
      "lon": -77.0000881
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Grazie Nonna",
    "address": "1100 15th St NW, Washington, DC  20005, United States",
    "location": {
      "lat": 38.9042968,
      "lon": -77.0352845
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "Spa Logic",
    "address": "1721 Connecticut Ave NW, Washington, DC  20009, United States",
    "location": {
      "lat": 38.913265,
      "lon": -77.045313
    },
    "category": "Beauty",
    "category2": "Spas"
  },
  {
    "name": "Decades",
    "address": "1219 Connecticut Avenue NW, Washington, DC 20036, United States",
    "location": {
      "lat": 38.906513,
      "lon": -77.04126
    },
    "category": "Nightlife",
    "category2": "Nightlife"
  },
  {
    "name": "Swingers Dupont Circle",
    "address": "1330 19th St NW, Washington, DC  20036, United States",
    "location": {
      "lat": 38.9084408,
      "lon": -77.0437496
    },
    "category": "MiniGolf",
    "category2": "Nightlife"
  },
  {
    "name": "St. Arnold's Mussel Bar",
    "address": "1827 Jefferson Pl NW, Washington, DC  20036, United States",
    "location": {
      "lat": 38.9065335,
      "lon": -77.0427796
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "The White House",
    "address": "1600 Pennsylvania Ave NW, Washington, DC 20500-0003, United States",
    "location": {
      "lat": 38.8976817,
      "lon": -77.036588
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "Enterprise Golf Course",
    "address": "2802 Enterprise Rd, Mitchellville, MD 20721, United States",
    "location": {
      "lat": 38.928356,
      "lon": -76.81691
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Live-K",
    "address": "730 Maine Ave SW, Washington, DC  20024, United States",
    "location": {
      "lat": 38.8788022,
      "lon": -77.0234163
    },
    "category": "Nightlife",
    "category2": "Nightlife"
  },
  {
    "name": "Washington Monument",
    "address": "2 15th St NW, Washington, DC  20024, United States",
    "location": {
      "lat": 38.88943,
      "lon": -77.0353955
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "World War II Memorial",
    "address": "1750 Independence Ave SW, Washington, DC 20024, United States",
    "location": {
      "lat": 38.8893877,
      "lon": -77.0405209
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "East Potomac Golf Course",
    "address": "972 Ohio Dr SW, Washington, DC  20024, United States",
    "location": {
      "lat": 38.8745842,
      "lon": -77.0267934
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Thomas Jefferson Memorial",
    "address": "16 E Basin Dr SW, Washington, DC 20024, United States",
    "location": {
      "lat": 38.8813621,
      "lon": -77.0365351
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "Vietnam Veterans Memorial",
    "address": "5 Henry Bacon Dr NW, Washington, DC 20002, United States",
    "location": {
      "lat": 38.8912541,
      "lon": -77.0477146
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "Martin Luther King, Jr. Memorial",
    "address": "1850 W Basin Dr SW, Washington, DC  20024, United States",
    "location": {
      "lat": 38.8862185,
      "lon": -77.0442197
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "il Canale",
    "address": "1065 31st St NW, Washington, DC  20007, United States",
    "location": {
      "lat": 38.9045021,
      "lon": -77.0609403
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "Lincoln Memorial",
    "address": "2 Lincoln Memorial Cir NW, Washington, DC  20037, United States",
    "location": {
      "lat": 38.889218,
      "lon": -77.050178
    },
    "category": "Landmark",
    "category2": "Historical Sites"
  },
  {
    "name": "The Sovereign",
    "address": "1206 Wisconsin Ave NW, Washington, DC  20007, United States",
    "location": {
      "lat": 38.9055479,
      "lon": -77.063183
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "Filomena Ristorante",
    "address": "1063 Wisconsin Ave NW, Washington, DC  20007, United States",
    "location": {
      "lat": 38.9044418,
      "lon": -77.0626614
    },
    "category": "Restaurant",
    "category2": "Italian food"
  },
  {
    "name": "Bowie Golf Club",
    "address": "7420 Laurel Bowie Rd\nBowie, MD  20715\nUnited States",
    "location": {
      "lat": 38.9960736,
      "lon": -76.7622685
    },
    "category": "Golf",
    "category2": "Golf"
  },
  {
    "name": "Et Voila",
    "address": "5120 MacArthur Blvd NW, Washington, DC  20016, United States",
    "location": {
      "lat": 38.925407,
      "lon": -77.102364
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "Woodhouse Spa",
    "address": "2 Paseo Dr, Rockville, MD  20852, United States",
    "location": {
      "lat": 39.0426612,
      "lon": -77.112211
    },
    "category": "Spa",
    "category2": "Spas"
  },
  {
    "name": "Bowie Town Center",
    "address": "15606 Emerald Way, Bowie, MD  20716, United States",
    "location": {
      "lat": 38.9442596,
      "lon": -76.7347193
    },
    "category": "Store",
    "category2": "shopping"
  },
  {
    "name": "Mannequin Pis",
    "address": "18064 Georgia Ave, Olney, MD  20832, United States",
    "location": {
      "lat": 39.1524126,
      "lon": -77.0676249
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "Autobahn Indoor Speedway",
    "address": "8251 Preston Ct, Ste F, Jessup, MD  20794, United States",
    "location": {
      "lat": 39.1497871,
      "lon": -76.7937002
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "K1 Speed",
    "address": "8251 Preston Ct, Jessup, MD  20794, United States",
    "location": {
      "lat": 39.1497871,
      "lon": -76.7937002
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "Crofton Go-Kart Raceway",
    "address": "1050 State Route 3 South, Gambrills, MD 21054, United States",
    "location": {
      "lat": 39.0282504,
      "lon": -76.6891592
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "United Karting",
    "address": "7206 Ridge Rd, Hanover, MD  21076, United States",
    "location": {
      "lat": 39.1798945,
      "lon": -76.7105019
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "Freetjes",
    "address": "1448 Light St, Baltimore, MD  21230, United States",
    "location": {
      "lat": 39.2729366,
      "lon": -76.6120387
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "Cock and Bowl",
    "address": "302 Poplar Alley, Occoquan, VA 22125, United States",
    "location": {
      "lat": 38.6838156,
      "lon": -77.2610078
    },
    "category": "Restaurant",
    "category2": "Belgian food"
  },
  {
    "name": "The Brewer's Art",
    "address": "1106 N Charles St, Baltimore, MD  21201, United States",
    "location": {
      "lat": 39.3027715,
      "lon": -76.6163875
    },
    "category": "Brewery",
    "category2": "Belgian food"
  },
  {
    "name": "Autobahn Indoor Speedway & Events",
    "address": "45448 E Severn Way, Unit 150, Sterling, VA 20166, United States",
    "location": {
      "lat": 39.0214199,
      "lon": -77.427635
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "K1 Speed",
    "address": "45448 E Severn Way, Sterling, VA  20166, United States",
    "location": {
      "lat": 39.0214199,
      "lon": -77.427635
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "Autobahn Indoor Speedway & Events",
    "address": "8415 Kelso Dr, Essex, MD 21221, United States",
    "location": {
      "lat": 39.326625,
      "lon": -76.4848801
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "K1 Speed",
    "address": "8300 Sudley Rd, Unit A5, Manassas, VA  20109, United States",
    "location": {
      "lat": 38.7720027,
      "lon": -77.5036097
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "Go Kart Track",
    "address": "10907 Pulaski Hwy, White Marsh, MD  21162, United States",
    "location": {
      "lat": 39.3819961,
      "lon": -76.4245172
    },
    "category": "GoKart",
    "category2": "Go Kart"
  },
  {
    "name": "Aquatic Sports",
    "address": "10803 SW Barbur Blvd, Portland, OR  97219, United States",
    "location": {
      "lat": 45.4473683,
      "lon": -122.7296765
    },
    "category": "Aquatic Sports",
    "category2": "Aquatic Sports"
  }
]

# Creating a DataFrame for places
places_df = pd.DataFrame(places_data)
places_df['combined_category'] = places_df.apply(lambda row: [row['category'], row['category2']], axis=1)

# Print the places DataFrame
places_df.head()

,name,address,location,category,category2,combined_category
0,Potomac Pizza,"7777 Baltimore Ave, College Park, MD 20740, U...","{'lat': 38.9873178, 'lon': -76.9356036}",Restaurant,Italian food,"[Restaurant, Italian food]"
1,The Spa at The Hotel at the University of Mary...,"7777 Baltimore Ave, FL 4, College Park, MD 20...","{'lat': 38.9869586, 'lon': -76.935151}",Spa,Spas,"[Spa, Spas]"
2,The Hall CP,"4656 Hotel Drive, College Park, MD 20742, Unit...","{'lat': 38.9861878, 'lon': -76.9336134}",Restaurant,Nightlife,"[Restaurant, Nightlife]"
3,Dog Haus Biergarten,"7401 Baltimore Ave, Unit 1A, College Park, MD ...","{'lat': 38.9812184, 'lon': -76.9375926}",Nightlife,Nightlife,"[Nightlife, Nightlife]"
4,Terrapins Turf,"4410 Knox Rd, College Park, MD 20740, United ...","{'lat': 38.9810151, 'lon': -76.9384146}",Nightlife,Nightlife,"[Nightlife, Nightlife]"


In [50]:
#Themes
# Themes with relevant categories

romantic_date = [
    "Cafe", "Restaurant", "Bakery", "AmusementPark", "Beach", "Winery", 
    "Theater", "MovieTheater", "Park", "Zoo", "Aquarium", "Store", 
    "MiniGolf", "Bowling", "MusicVenue", "Store", "Mall"
]

family_outing = [
    "AmusementPark", "Zoo", "Aquarium", "Park", "Playground", "MovieTheater", 
    "Museum", "NationalPark", "Beach", "Campground", "FoodMarket"
]

outdoor_adventure = [
    "NationalPark", "Park", "Beach", "Hiking", "Kayaking", "Fishing", 
    "Golf", "MiniGolf", "RockClimbing", "RVPark", "SkatePark", "Skating", 
    "Skiing", "Surfing", "Swimming", "Tennis", "Volleyball"
]

educational_trip = [
    "Museum", "Library", "Aquarium", "NationalPark", "Planetarium", "Zoo", 
    "University", "Landmark", "NationalMonument", "ReligiousSite"
]

night_out = [
    "Nightlife", "Brewery", "Restaurant", "MovieTheater", "Theater", 
    "MusicVenue", "Casino", "Bar", "Store", "Winery"
]

relaxation_and_wellness = [
    "Beach", "Spa", "FitnessCenter", "Park", "Yoga", "MeditationCenter"
]

sports_and_fitness = [
    "Stadium", "FitnessCenter", "Golf", "Tennis", "Basketball", "Soccer", 
    "Baseball", "Swimming", "Volleyball", "Bowling", "RockClimbing", 
    "Hiking", "Kayaking", "Surfing", "Skating", "Skiing", "SkatePark"
]

shopping_spree = [
    "Store", "FoodMarket", "Mall", "Pharmacy"
]

kids_fun_day = [
    "AmusementPark", "Zoo", "Aquarium", "Park", "Playground", "MovieTheater", 
    "MiniGolf", "Bowling", "Fairground", "GoKart"
]

historical_and_cultural_exploration = [
    "Museum", "Castle", "Fortress", "Landmark", "NationalMonument", 
    "ReligiousSite", "Planetarium", "Fairground", "ConventionCenter"
]

Vacation = [
    "Hotel", "Beach", "NationalPark", "Park", "Winery", "Campground", 
    "Marina", "Skiing", "RVPark", "Store"
]

food_and_drink = [
    "Restaurant", "Cafe", "Bakery", "Brewery", "Winery", "FoodMarket"
]



In [51]:
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
import numpy as np
import torch

# Initialize encoders
mlb = MultiLabelBinarizer()
le_name = LabelEncoder()
le_address = LabelEncoder()
le_user = LabelEncoder()

# Fit the MultiLabelBinarizer on both places and user preferences
all_categories = list(places_df['combined_category'].explode().unique()) + list(df_user_profile['combined_preferences'].explode().unique())
mlb.fit([all_categories])

# Transform the combined categories
places_encoded = mlb.transform(places_df['combined_category'])
user_encoded = mlb.transform(df_user_profile['combined_preferences'])

# Encode the name and address columns
places_df['name_encoded'] = le_name.fit_transform(places_df['name'])
places_df['address_encoded'] = le_address.fit_transform(places_df['address'])
ratings_df['user_encoded'] = le_user.fit_transform(ratings_df['User'])

# Extract latitude and longitude
places_df['lat'] = places_df['location'].apply(lambda x: x['lat'])
places_df['lon'] = places_df['location'].apply(lambda x: x['lon'])

# Create the final places feature matrix
places_features = np.hstack([places_df[['name_encoded', 'address_encoded', 'lat', 'lon']].values, places_encoded])
user_features = np.hstack([user_encoded, df_user_profile[['Budget']].values])

places_features_df = pd.DataFrame(places_features)
user_features_df = pd.DataFrame(user_features)

print("Encoded Places DataFrame:")
print(places_features_df)
print("\nEncoded User DataFrame:")
print(user_features_df)


Encoded Places DataFrame:
      0     1          2           3    4    5    6    7    8    9   ...   18  \
0   57.0  72.0  38.987318  -76.935604  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
1   73.0  73.0  38.986959  -76.935151  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2   69.0  43.0  38.986188  -76.933613  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0   
3   20.0  69.0  38.981218  -76.937593  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0   
4   66.0  40.0  38.981015  -76.938415  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0   
..   ...   ...        ...         ...  ...  ...  ...  ...  ...  ...  ...  ...   
85  40.0  41.0  39.021420  -77.427635  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0   
86   5.0  80.0  39.326625  -76.484880  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0   
87  40.0  79.0  38.772003  -77.503610  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0   
88  31.0   6.0  39.381996  -76.424517  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0   
89   2.0   5.0  45.447368 -122.729676  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

 

In [52]:
# Convert features to tensors
places_tensor = torch.tensor(places_features, dtype=torch.float32)
user_tensor = torch.tensor(user_features, dtype=torch.float32)

places_tensor.shape, user_tensor.shape


(torch.Size([90, 28]), torch.Size([4, 25]))

In [53]:
places_tensor.shape, user_tensor.shape

(torch.Size([90, 28]), torch.Size([4, 25]))

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

# Create user-item interaction matrix
user_item_matrix = ratings_df.pivot(index='user_encoded', columns='name', values='Rating').fillna(0)

# Collaborative Filtering: Predict missing ratings using cosine similarity
def predict_ratings(user_item_matrix, epsilon=1e-9):
    # Compute user similarity matrix
    user_similarity = cosine_similarity(user_item_matrix)
    np.fill_diagonal(user_similarity, 0)
    
    # Predict ratings
    denominator = np.array([np.abs(user_similarity).sum(axis=1)]).T + epsilon
    ratings_pred = user_similarity.dot(user_item_matrix) / denominator
    
    # Normalize predictions
    ratings_pred = (ratings_pred - ratings_pred.min()) / (ratings_pred.max() - ratings_pred.min())
    
    return ratings_pred

# Calculate predicted ratings for all users
predicted_ratings = predict_ratings(user_item_matrix)

/var/folders/96/hb5vtsbx3kx_z_pktn0xx0ym0000gn/T/ipykernel_51367/1117007371.py:17: RuntimeWarning: invalid value encountered in divide
  ratings_pred = (ratings_pred - ratings_pred.min()) / (ratings_pred.max() - ratings_pred.min())


In [55]:
import torch
import torch.nn as nn

class ContentModel(nn.Module):
    def __init__(self, input_dim):
        super(ContentModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc4(x)
        return x

def pad_tensor(tensor, target_size):
    current_size = tensor.shape[1]
    if current_size < target_size:
        padding = torch.zeros(tensor.shape[0], target_size - current_size)
        return torch.cat([tensor, padding], dim=1)
    return tensor

def cosine_similarity_tensor(tensor1, tensor2):
    max_features = max(tensor1.shape[1], tensor2.shape[1])
    tensor1 = pad_tensor(tensor1, max_features)
    tensor2 = pad_tensor(tensor2, max_features)
    
    tensor1 = tensor1.unsqueeze(1)
    tensor2 = tensor2.unsqueeze(0)
    
    dot_product = torch.sum(tensor1 * tensor2, dim=2)
    norm1 = torch.norm(tensor1, dim=2)
    norm2 = torch.norm(tensor2, dim=2)
    similarity = dot_product / (norm1 * norm2 + 1e-9)
    return similarity

# Determine the maximum number of features
max_features = max(user_tensor.shape[1], places_tensor.shape[1])

# Pad both tensors to have the same number of features
user_tensor_padded = pad_tensor(user_tensor, max_features)
places_tensor_padded = pad_tensor(places_tensor, max_features)

# Create the combined tensor
user_tensor_repeated = user_tensor_padded.repeat_interleave(places_tensor_padded.shape[0], dim=0)
user_place_tensor = torch.cat((user_tensor_repeated, places_tensor_padded.repeat(user_tensor_padded.shape[0], 1)), dim=1)

# Update the input dimension for the model
input_dim = user_place_tensor.shape[1]

# Create the model
model = ContentModel(input_dim)

print("user_place_tensor shape:", user_place_tensor.shape)
print("Model input dimension:", input_dim)

# Calculate similarity
similarity = cosine_similarity_tensor(user_tensor_padded, places_tensor_padded)
target_tensor = similarity.view(-1, 1)

print("target_tensor shape:", target_tensor.shape)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(user_place_tensor)
    loss = criterion(output, target_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

user_place_tensor shape: torch.Size([360, 56])
Model input dimension: 56
target_tensor shape: torch.Size([360, 1])
Epoch 100/1000, Loss: 0.0002
Epoch 200/1000, Loss: 0.0001
Epoch 300/1000, Loss: 0.0000
Epoch 400/1000, Loss: 0.0000
Epoch 500/1000, Loss: 0.0000
Epoch 600/1000, Loss: 0.0000
Epoch 700/1000, Loss: 0.0000
Epoch 800/1000, Loss: 0.0000
Epoch 900/1000, Loss: 0.0000
Epoch 1000/1000, Loss: 0.0000


In [56]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

print("user_place_tensor shape:", user_place_tensor.shape)
print("places_df shape:", places_df.shape)

# Make predictions with content based model
model.eval()
with torch.no_grad():
    content_predictions = model(user_place_tensor)
    content_predictions = content_predictions.cpu().numpy().flatten()

print("content_predictions shape:", content_predictions.shape)

# Reshape content_predictions to match the number of places
num_users = user_tensor.shape[0]
num_places = places_df.shape[0]
content_predictions = content_predictions.reshape(num_users, num_places)

# Average predictions across users
avg_content_predictions = np.mean(content_predictions, axis=0)

# Combine predictions with places for better understanding
recommendations = places_df.copy()
recommendations['content_score'] = avg_content_predictions

# Decode the name and address columns
recommendations['name'] = le_name.inverse_transform(recommendations['name_encoded'])
recommendations['address'] = le_address.inverse_transform(recommendations['address_encoded'])

# Assign collaborative scores based on ratings
def get_collab_score(name, user_idx):
    if name in user_item_matrix.columns and user_idx < predicted_ratings.shape[0]:
        score = predicted_ratings[user_idx][user_item_matrix.columns.get_loc(name)]
        return score if not np.isnan(score) else 0
    return 0

# Let's assume we want recommendations for User2 (index 1)
user_idx = 1

# Assign collaborative score for each place
recommendations['collab_score'] = recommendations['name'].apply(lambda x: get_collab_score(x, user_idx))

# Normalize both scores for combination
def safe_normalize(series):
    min_val = series.min()
    max_val = series.max()
    if min_val == max_val:
        return pd.Series([0.5] * len(series))
    return (series - min_val) / (max_val - min_val)

recommendations['content_score'] = safe_normalize(recommendations['content_score'])
recommendations['collab_score'] = safe_normalize(recommendations['collab_score'])



# Check the distribution of collaborative scores
print("Collaborative score distribution:")
print(recommendations['collab_score'].describe())

# If all scores are the same, add some randomness
if recommendations['collab_score'].nunique() == 1:
    recommendations['collab_score'] += np.random.uniform(0, 0.1, size=len(recommendations))

# Normalize collaborative scores
recommendations['collab_score'] = (recommendations['collab_score'] - recommendations['collab_score'].min()) / \
                                  (recommendations['collab_score'].max() - recommendations['collab_score'].min())


# Combine content-based and collaborative scores with a weighting factor
alpha = 0.7  # Weight for content-based model
beta = 0.3   # Weight for collaborative model

recommendations['final_score'] = (alpha * recommendations['content_score'] + 
                                  beta * recommendations['collab_score'])

# Sort recommendations by final score
recommendations = recommendations.sort_values(by='final_score', ascending=False)

# Display the top recommendations
print("\nTop Recommendations for User2:")
print(recommendations[['name', 'address', 'category', 'content_score', 'collab_score', 'final_score']].head(10))

user_place_tensor shape: torch.Size([360, 56])
places_df shape: (90, 10)
content_predictions shape: (360,)
Collaborative score distribution:
count    90.0
mean      0.5
std       0.0
min       0.5
25%       0.5
50%       0.5
75%       0.5
max       0.5
Name: collab_score, dtype: float64

Top Recommendations for User2:
                           name  \
87                     K1 Speed   
58      St. Arnold's Mussel Bar   
82                Cock and Bowl   
28    Parkway Spa Asian Massage   
41  Iron Age Korean Steak House   
34             Bryant Street NE   
60       Enterprise Golf Course   
68                    il Canale   
37      Sligo Creek Golf Course   
27                Go Kart Track   

                                              address    category  \
87  8300 Sudley Rd, Unit A5, Manassas, VA  20109, ...      GoKart   
58  1827 Jefferson Pl NW, Washington, DC  20036, U...  Restaurant   
82  302 Poplar Alley, Occoquan, VA 22125, United S...  Restaurant   
28  5507 Landover 